In [64]:
import random
import sys
import torch
import torch.nn as nn
from torch import cuda
from torch import optim
gpu_id = None

In [65]:
# model
class fittingRNN(nn.Module):
    def __init__(self, in_neurons = 2, hidden_neurons = 10, out_neurons = 1):
        super(fittingRNN, self).__init__()
        self.hidden_neurons = hidden_neurons
        self.W_x_h = nn.Linear(in_neurons, hidden_neurons)
        self.W_h_h = nn.Linear(hidden_neurons, hidden_neurons)
        self.W_h_y = nn.Linear(hidden_neurons, out_neurons)

        if gpu_id is not None:
            self.device = torch.device(gpu_id)
            self.to(self.device)
        else:
            self.device = torch.device('cpu')

        self.reset_state()

    def reset_state(self):
        self.h = torch.zeros(1, self.hidden_neurons).to(self.device)

    def forward(self, cur):
        self.h = torch.tanh(self.W_x_h(cur.to(self.device)) + self.W_h_h(self.h))
        y = self.W_h_y(self.h)
        return y

In [71]:
dataset = []
for _ in range(200):
    res = [[1, 0]]
    for i in range(200):
        nex = [random.randint(0, 1), res[-1][1] + random.uniform(-0.2, 0.2)]
        if res[-1][0] == 1:
            nex[1] += 1
        else:
            nex[1] -= 1
        res.append(nex)
    dataset.append(res)
print(dataset)

[[[1, 0], [1, 1.141266082402269], [0, 2.1380012781802296], [0, 0.9546940805693778], [1, -0.07369607496413522], [0, 0.7671085151638821], [1, -0.24265125029543433], [1, 0.7467458890445544], [0, 1.8710493202092466], [1, 0.9716215077610639], [0, 1.9200568225193095], [0, 1.1108243636409667], [0, 0.15971439286930833], [0, -0.833176206680665], [0, -1.9118866663988017], [1, -3.0684811899912656], [0, -2.2291098187028395], [1, -3.413601254675025], [1, -2.4563005899475834], [0, -1.3709682147261097], [1, -2.263600292026875], [0, -1.3965161350079578], [0, -2.476749596752102], [0, -3.4726387996457233], [0, -4.528611171540501], [0, -5.605944131864774], [1, -6.569063965213826], [1, -5.5673700963000226], [1, -4.619669894543319], [1, -3.651425615098124], [0, -2.503014339283353], [0, -3.4444531877774742], [1, -4.541974503083791], [0, -3.6937632602394723], [0, -4.51910986124342], [1, -5.605259489366462], [1, -4.594838114946587], [0, -3.595005876759198], [1, -4.414487463991055], [0, -3.4754992430332097], [

In [72]:
# train
model = fittingRNN()
loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters())

epoch_num = 200
for epoch in range(epoch_num):
    print("{0} / {1} epoch start.".format(epoch + 1, epoch_num))
    sum_loss = 0.0
    for i, data in enumerate(dataset):
        model.reset_state()
        optimizer.zero_grad()
        accum_loss = None
        for cur, next in zip(data, data[1:] + [[0, 0]]):
            cur = torch.tensor([float(cur[0]), float(cur[1])])
            next = torch.tensor([float(next[1])]).unsqueeze(-1)
            out = model(cur)
            loss = loss_fn(out, next.to(out.device))
            accum_loss = loss if accum_loss is None else accum_loss + loss
        accum_loss.backward()
        optimizer.step()
        sum_loss += float(accum_loss.data.cpu())
        if (i + 1) % 100 == 0:
            print("{0} / {1} sentences finished.".format(i + 1, len(dataset)))
    print("mean loss = {0}.".format(sum_loss))

    model_file = "../trained_model/fitting_model_rnn_" + str(epoch + 1) + ".model"
    torch.save(model.state_dict(), model_file)

1 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 3476186.574951172.
2 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 2756090.733444214.
3 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 2236751.6707077026.
4 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 1855713.2785282135.
5 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 1568101.4778289795.
6 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 1342923.272075653.
7 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 1162513.3315224648.
8 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 1015690.1934270859.
9 / 200 epoch start.
100 / 200 sentences finished.
200 / 200 sentences finished.
mean loss = 894449

In [76]:
# test
model = fittingRNN()
model.load_state_dict(torch.load("../trained_model/fitting_model_rnn_200.model"))
model.eval()

data = [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# answer = [0.0, 1.0, 2.0, 3.0, 2.0, 3.0, 4.0, 3.0, 2.0, 1.0, 0.0, -1.0]
y = torch.tensor(0.0)
x = torch.tensor([data[0], y])
print(y)
for i in range(1, len(data)):
    y = model(x)
    print(y)
    x = torch.tensor([data[i], y])

tensor(0.)
tensor([[1.0388]], grad_fn=<AddmmBackward0>)
tensor([[2.1129]], grad_fn=<AddmmBackward0>)
tensor([[3.1229]], grad_fn=<AddmmBackward0>)
tensor([[2.1274]], grad_fn=<AddmmBackward0>)
tensor([[3.1454]], grad_fn=<AddmmBackward0>)
tensor([[4.1289]], grad_fn=<AddmmBackward0>)
tensor([[3.1206]], grad_fn=<AddmmBackward0>)
tensor([[2.1681]], grad_fn=<AddmmBackward0>)
tensor([[1.2325]], grad_fn=<AddmmBackward0>)
tensor([[0.3116]], grad_fn=<AddmmBackward0>)
tensor([[-0.6067]], grad_fn=<AddmmBackward0>)
